# Import Libraries
Importing libraries are necessary for any project.
We're going to import commonly used Webscrapping and Data Analysis libraries. Make sure all libraries are installed.

In [1]:
import requests # for getting web contents
from bs4 import BeautifulSoup # for scraping web contents
import pandas as pd # for data analysis
import numpy as np
from progressbar import ProgressBar, Percentage, Bar

# URL
For web scrapping it's important to have url.

In [2]:
# link from where you want to scrap data
base_url = 'https://myneta.info/loksabha2004/'

# get web data
page = requests.get(base_url)

# parse web data
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
# find the table
# our trageted table is last

# this link have lists of links so first get all the links, all links stored in table so getting last table
table = soup.find_all('table')[-1]

# links stored in div class='items'
divs = table.find_all('div', class_="items")

links = []
cities = []

for div in divs:
    for a in div.find_all('a', href=True):
        link = a['href']
        if '/' in link:
            link = link.split('/')[-1:]
            links.append(link)
        elif '/' not in link:
            links.append(link)
        city = a.text.strip()
        cities.append(city)

In [4]:
# function to scrape values from each url
def url_scrraper(url,city):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find_all('table', id='table1')
    if len(table) != 0:
        for tab in table:
            thead = tab.find_all('th')
    
            for i in thead:
                headings.append(i.text)
            headings.insert(-1,'Constituency')
    
            html_text = [tr for tr in tab.find_all('tr')][1:]
    
            for text in html_text:
                td = [cells for cells in text.find_all('td')]
                if len(td)!= 0 :
                    name = td[0].find('a').get_text().title()
                    party = td[1].get_text()
                    case = td[2].get_text()
                    edu = td[3].get_text()
                    age = td[4].get_text()
                    ta = td[5].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
                    la = td[6].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
                    if(ta == 'Nil'):
                        ta = np.NaN
                    if(la == 'Nil'):
                        la = np.NaN
                    row = [name, party, case, edu, age, ta, la]
                    row.insert(-1,city)
                    content.append(row)

In [5]:
content = []

pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=len(cities)).start()

for i in range(len(cities)):
    pbar.update(i+1)
    headings = []
    if type(links[i]) == list:
        url = base_url + links[i][0]
    else:
        url = base_url + links[i]
    url_scrraper(url, cities[i])

pbar.finish()

100%|#########################################################################|


In [6]:
# save contents in a dataframe
data = pd.DataFrame(content[:], columns=headings)

In [7]:
# link from where you want to scrap data
winner_url = 'index.php?action=show_winners&sort=default'

# get web data
page = requests.get(base_url+winner_url)

# parse web data
soup = BeautifulSoup(page.content, "html.parser")

In [8]:
table = soup.find_all('table')[-1]
thead = [th for th in table.find_all('th')]
winner_headings = []

for i in thead:
    winner_headings.append(i.text.strip())

winner_content = []
html_text = [tr for tr in table.find_all('tr')][1:]
    
for text in html_text:
    td = [cells for cells in text.find_all('td')]
    if len(td)!= 0 :
        name = td[1].find('a').get_text().title()
        city = td[2].get_text().title()
        party = td[3].get_text().strip()
        case = td[4].get_text().strip()
        edu = td[5].get_text().strip()
        ta = td[6].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        la = td[7].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        if(ta == 'Nil'):
            ta = np.NaN
        if(la == 'Nil'):
            la = np.NaN
        row = [name, city, party, case, edu, ta, la]
        winner_content.append(row)

In [9]:
# save contents in a dataframe
winner_data = pd.DataFrame(winner_content[:], columns=winner_headings[1:])

In [10]:
winner_data = winner_data.rename(columns={'Constituency ∇':'Constituency', 'Criminal Case':'Criminal Cases'})

In [11]:
winner_data['Winner'] = '1'

In [12]:
women_url = "index.php?action=summary&subAction=women_candidate&sort=candidate#summary"
page = requests.get(base_url + women_url)
soup = BeautifulSoup(page.content, 'html.parser')

women_headings = []

table = soup.find_all('table')[-1]
thead = [th for th in table.find_all('th')]

for i in thead:
    women_headings.append(i.text.strip())

women_content = []
html_text = [tr for tr in table.find_all('tr')][1:]
    
for text in html_text:
    td = [cells for cells in text.find_all('td')]
    if len(td)!= 0 :
        name = td[1].find('a').get_text().title()
        city = td[2].get_text().title()
        party = td[3].get_text().strip()
        case = td[4].get_text().strip()
        edu = td[5].get_text().strip()
        ta = td[6].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        la = td[7].contents[0].strip().replace(u'Rs\xa0',u'').replace(',','')
        if(ta == 'Nil'):
            ta = np.NaN
        if(la == 'Nil'):
            la = np.NaN
        row = [name, city, party, case, edu, ta, la]
        women_content.append(row)

In [13]:
# save contents in a dataframe
women_data = pd.DataFrame(women_content[:], columns=women_headings[1:])

In [14]:
women_data = women_data.rename(columns={'Candidate∇':'Candidate', 'Criminal Case':'Criminal Cases'})

In [15]:
women_data['Gender'] = 'F'

In [16]:
final_data = pd.merge(left=data, right=winner_data, 
                      how ='left', on=['Candidate', 'Criminal Cases', \
                                       'Constituency', 'Party', 'Education', \
                                       'Liabilities','Total Assets'])

final_data = pd.merge(left=final_data, right=women_data, 
                      how ='left', on=['Candidate', 'Criminal Cases', \
                                       'Constituency', 'Party', 'Education', \
                                       'Liabilities','Total Assets'])

In [17]:
final_data

,Candidate,Party,Criminal Cases,Education,Age,Total Assets,Constituency,Liabilities,Winner,Gender
0,Manoranjan Bhakta,INC,0,Post Graduate,65,5926740,Andaman & Nicobar Islands,272061,1,NaN
1,A K Biswas,BSP,0,Graduate,61,7876500,Andaman & Nicobar Islands,65000,NaN,NaN
2,Asit Baran Dutta,IND,0,Graduate,50,3070000,Andaman & Nicobar Islands,0,NaN,NaN
3,Bishnu Pada Ray,BJP,0,Graduate,54,1250619,Andaman & Nicobar Islands,128710,NaN,NaN
4,Deepak Biswas,IND,0,Not Given,28,3000,Andaman & Nicobar Islands,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3637,Susmita Bauri,CPM,0,Graduate Professional,29,33000,Vishnupur,10000,1,F
3638,Achintya Majhi,INC,0,Graduate,38,67264,Vishnupur,50050,NaN,NaN
3639,Ajoy Bauri,BSP,0,Not Given,40,35000,Vishnupur,0,NaN,NaN
3640,Janardan Saha,AITC,0,Post Graduate,37,233612,Vishnupur,14000,NaN,NaN


In [18]:
final_data['Winner'] = final_data['Winner'].fillna('0')
final_data['Gender'] = final_data['Gender'].fillna('M')

In [19]:
final_data.Winner = final_data.Winner.astype('object')
final_data['Criminal Cases'] = final_data['Criminal Cases'].astype('int')

# Data Analysis

## Check Data Shape

In [20]:
# data shape
print(final_data.shape)

(3642, 10)


## Look at Example Records

In [21]:
# check few top rows of data
final_data.head()

,Candidate,Party,Criminal Cases,Education,Age,Total Assets,Constituency,Liabilities,Winner,Gender
0,Manoranjan Bhakta,INC,0,Post Graduate,65,5926740,Andaman & Nicobar Islands,272061,1,M
1,A K Biswas,BSP,0,Graduate,61,7876500,Andaman & Nicobar Islands,65000,0,M
2,Asit Baran Dutta,IND,0,Graduate,50,3070000,Andaman & Nicobar Islands,0,0,M
3,Bishnu Pada Ray,BJP,0,Graduate,54,1250619,Andaman & Nicobar Islands,128710,0,M
4,Deepak Biswas,IND,0,Not Given,28,3000,Andaman & Nicobar Islands,0,0,M


## Descriptive Statistics

In [22]:
# getting Generate descriptive statistics of data. Generate descriptive statistics include count, mean, std, min_value, 25%, 50%, 75%, max_value
final_data.describe()

,Criminal Cases
count,3642.000000
mean,0.373421
std,1.809383
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,36.000000


## Summary of data-type, columns, non-null values, memory usage.

In [23]:
# data info
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3642 entries, 0 to 3641
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Candidate       3642 non-null   object
 1   Party           3642 non-null   object
 2   Criminal Cases  3642 non-null   int64 
 3   Education       3642 non-null   object
 4   Age             3642 non-null   object
 5   Total Assets    3382 non-null   object
 6   Constituency    3642 non-null   object
 7   Liabilities     3642 non-null   object
 8   Winner          3642 non-null   object
 9   Gender          3642 non-null   object
dtypes: int64(1), object(9)
memory usage: 313.0+ KB


## Column labels

In [24]:
# get the column labels of the data.
final_data.columns

Index(['Candidate', 'Party', 'Criminal Cases', 'Education', 'Age',
       'Total Assets', 'Constituency', 'Liabilities', 'Winner', 'Gender'],
      dtype='object')

# Save Data into CSV

In [25]:
# save data
final_data.to_csv('LokSabha2004.csv', index=False)